In [1]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/bruno/Desktop/coarse-discourse-validation


In [2]:
import pandas as pd
import json

In [3]:
filepath = os.path.join(PROJ_ROOT, 'data', 'raw', 'coarse_discourse_dump_reddit.json')

raw_df = pd.read_json(filepath, lines=True) # 'lines' so pandas reads each line of file as a json object

# Converts 'is_self_post' column to boolean
raw_df['is_self_post'] = raw_df['is_self_post'].fillna(0).astype('bool')

raw_df[raw_df['is_self_post']!= True].head()

raw_df.head()

,is_self_post,posts,subreddit,title,url
0,True,"[{'id': 't3_1bx6qw', 'annotations': [{'link_to...",100movies365days,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...
1,True,"[{'id': 't3_omv7p', 'annotations': [{'link_to_...",100sets,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...
2,True,"[{'id': 't3_259tbh', 'annotations': [{'link_to...",1200isplenty,122cal black currant cheesecake!,https://www.reddit.com/r/1200isplenty/comments...
3,True,"[{'id': 't3_16h61h', 'annotations': [{'link_to...",1911,Need help finding a Springfield!,https://www.reddit.com/r/1911/comments/16h61h/...
4,True,"[{'id': 't3_35igzp', 'annotations': [{'link_to...",1911,Help with a possible trade?,https://www.reddit.com/r/1911/comments/35igzp/...


# Data preprocessing

To train our models, we want every post as a line in the final dataset. Note that we keep the features (but the url) of the thread so we don't discard any data for now.

In [4]:
dpproc = pd.DataFrame()

for i in range(1, len(raw_df)):
    thread = raw_df.iloc[i]
    dposts = pd.DataFrame(thread['posts'])
    dposts['is_self_post'] = thread['is_self_post']
    dposts['subreddit'] = thread['subreddit']
    dposts['thread_title'] = thread['title']

    dpproc = pd.concat([dpproc, dposts], ignore_index=True, sort=True)

dpproc.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,annotations,author,body,id,in_reply_to,is_first_post,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,True,none,announcement,NaN,100sets,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...
1,"[{'link_to_post': 't3_omv7p', 'main_type': 'el...",Keatonus,**January 16th 3 Sets:** \nWent out shopping w...,t1_c3igqif,t3_omv7p,NaN,True,t3_omv7p,elaboration,1.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN
2,"[{'link_to_post': 't1_c3igqif', 'main_type': '...",NaN,NaN,t1_c3imrkb,t1_c3igqif,NaN,True,t1_c3igqif,humor,2.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN
3,"[{'link_to_post': 't3_omv7p', 'main_type': 'ap...",answerorreply,"dude, these sets are awesome. You're doing gre...",t1_c3ij8z0,t3_omv7p,NaN,True,t3_omv7p,appreciation,1.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN
4,"[{'link_to_post': 't1_c3ij8z0', 'main_type': '...",Keatonus,"Thanks man! Yeah I'm trying to just keep the ""...",t1_c3inx9t,t1_c3ij8z0,NaN,True,t1_c3ij8z0,appreciation,2.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN


Here we can see that we get the same amount of posts as in 1.1 notebook.

In [5]:
len(dpproc)

116353

## Columns cleaning

### is_first_post

Now we verify that `is_first_post` column is useless.

In [8]:
dpproc[dpproc['in_reply_to'].isna()]['is_first_post'].value_counts()

True    9482
Name: is_first_post, dtype: int64

So we drop it.

In [9]:
dpproc.drop(columns=['is_first_post'], inplace=True)

Acknowledging that _NaN_ values at the `in_reply_to` column mean that it is the first post

### post_depth

In [14]:
dpproc['post_depth'].fillna(0).value_counts()

1.0     51262
2.0     25651
3.0     14320
0.0      9482
4.0      7228
5.0      3830
6.0      2086
7.0      1182
8.0       657
9.0       401
10.0      254
Name: post_depth, dtype: int64

In [15]:
dpproc['post_depth'] = dpproc['post_depth'].fillna(0)

### body

If `body` is _NaN_ then the post is useless to us.

In [18]:
dpproc.dropna(subset=['body'], inplace=True)

AttributeError: 'NoneType' object has no attribute 'head'

In [20]:
len(dpproc)

114437

This means a 1,65% loss.

### author

We'll assume that _NaN_ values at this column mean that the user was deleted.

In [23]:
dpproc[dpproc['author'].isna()].head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_title,url
6,"[{'link_to_post': 't3_omv7p', 'main_type': 'qu...",NaN,More updates!,t1_c3qodom,t3_omv7p,True,t3_omv7p,question,1.0,100sets,"Male, 23 years old. Going for 100 sets!",NaN
8,"[{'link_to_post': 't3_259tbh', 'main_type': 'a...",NaN,>very specific items that aren't available eve...,t1_chrzb0h,t3_259tbh,True,t3_259tbh,appreciation,1.0,1200isplenty,122cal black currant cheesecake!,NaN
11,"[{'link_to_post': 't1_chs0fj3', 'main_type': '...",NaN,"Oh, we have gelatin, but I've never seen it in...",t1_chs51e3,t1_chs0eqa,True,t1_chs0eqa,elaboration,3.0,1200isplenty,122cal black currant cheesecake!,NaN
15,"[{'link_to_post': 'none', 'main_type': 'questi...",NaN,I can't find them anywhere! I just want the mo...,t3_16h61h,NaN,True,none,question,0.0,1911,Need help finding a Springfield!,https://www.reddit.com/r/1911/comments/16h61h/...
21,"[{'link_to_post': 't1_c7w0wls', 'main_type': '...",NaN,"Oh hell no. Sorry, I linked to the wrong pisto...",t1_c7w1x38,t1_c7w0wls,True,t1_c7w0wls,answer,2.0,1911,Need help finding a Springfield!,NaN


In [27]:
dpproc[dpproc['author']=='deleted'].head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_title,url


In [32]:
dpproc['author'] = dpproc['author'].fillna('deleted')

## Final version

In [1]:
dpproc.head()

NameError: name 'dpproc' is not defined